## Imports and setups

In [ ]:
import tensorcircuit as tc
import numpy as np

tc.set_dtype("complex128")
tc.set_contractor("cotengra")
K = tc.set_backend("tensorflow")
import tensorflow as tf
import scipy
from functools import partial
from itertools import product

# the code work for both cpu and gpu

## Hamiltonian

In [ ]:
# 2D XXZ model

Lx = 4
Ly = 3
lattice = tc.templates.graphs.Grid2DCoord(Lx, Ly).lattice_graph(pbc=True)
h = tc.quantum.heisenberg_hamiltonian(lattice, hxx=1, hyy=1, hzz=0.8, hz=0.00000001)
e0 = scipy.sparse.linalg.eigsh(K.numpy(h), k=1, which="SA")[0]
print("exact gs", e0)

In [ ]:
# full Hamiltonian spectrum

es, u = np.linalg.eigh(K.numpy(K.real(K.to_dense(h))))

In [ ]:
# half-filling sector


@K.jit
def get_sumz(s):
    c = tc.Circuit(Lx * Ly, inputs=s)
    sumz = sum([c.expectation_ps(z=[j]) for j in range(Lx * Ly)])
    return K.real(sumz)


collect = []
for i in range(2 ** (Lx * Ly)):
    sumz = get_sumz(u[:, i])
    if K.abs(sumz) < 0.1:
        collect.append(i)
len(collect)

# overlap[collect] keeps only spectrum of the half-filling sector

## Imaginary-time evolved states

In [ ]:
# EE for ITES


@K.jit
def get_ee(beta):
    s = K.sum(
        [K.exp(-beta / 2 * es[i]) * u[:, i] for i in range(2 ** (Lx * Ly))], axis=0
    )
    target = s
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    ee = tc.quantum.entanglement_entropy(target, [0, 1, 4, 5, 8, 9])
    return ee

## MPS

In [ ]:
def get_energy(us):
    state = get_state(us)
    s = state / K.norm(state)
    c = tc.Circuit(Lx * Ly, inputs=s)
    energy = tc.templates.measurements.operator_expectation(c, h)
    return K.real(energy)


def get_state(us):
    # Lx*Ly, chi, chi, 2
    ns = []
    for i in range(Lx * Ly):
        ns.append(tc.gates.Gate(K.copy(us[i])))
    for i in range(Lx * Ly):
        ns[i][1] ^ ns[(i + 1) % (Lx * Ly)][0]
    m = tc.contractor(ns, output_edge_order=[n[2] for n in ns])
    s = K.reshape(m.tensor, [-1])
    return s


def get_fidelity(us, gs):
    state = get_state(us)
    s = state / K.norm(state)
    s = K.cast(s, tc.dtypestr)
    return -K.abs(tf.tensordot(s, gs, 1)) ** 2


vgf_fidelity = K.jit(K.value_and_grad(get_fidelity))
vgf_e = K.jit(K.value_and_grad(get_energy))

In [ ]:
# ITES training


def get_ites(beta, chi=16, steps=300):
    s = K.sum(
        [K.exp(-beta / 2 * es[i]) * u[:, i] for i in range(2 ** (Lx * Ly))], axis=0
    )
    target = s
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    target = K.cast(target, tc.dtypestr)
    lst = tf.keras.optimizers.schedules.ExponentialDecay(3e-3, 1000, 0.5)
    opt = K.optimizer(tf.keras.optimizers.Adam(lst))
    hist = []
    us = np.random.normal(scale=0.1, size=[Lx * Ly, chi, chi, 2])
    for i in range(steps):
        v, gs = vgf_fidelity(us, target)
        us = opt.update(gs, us)
        hist.append(v)
    state = get_state(us)
    state = state / K.norm(state)
    overlap0 = np.abs(K.numpy(target).reshape([1, -1]) @ u) ** 2
    overlap = np.abs(K.numpy(state).reshape([1, -1]) @ u) ** 2
    slope, intercept, r, p, slopee = scipy.stats.linregress(es, np.log(overlap))
    return (v, slope, intercept, r, p, slopee, overlap0, overlap, hist)

In [ ]:
def get_ites_bfgs(beta, chi=16, steps=300):
    s = K.sum(
        [K.exp(-beta / 2 * es[i]) * u[:, i] for i in range(2 ** (Lx * Ly))], axis=0
    )
    target = s
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    target = K.cast(target, tc.dtypestr)

    def loss_wrapper(vgf):
        def f(*args, **kws):
            v, gs = vgf(*args, **kws)
            f.loss.append(v)
            return v, gs

        f.loss = []
        return f

    vgf_scipy = tc.interfaces.scipy_optimize_interface(
        partial(get_fidelity, gs=target),
        jit=True,
        shape=[Lx * Ly, chi, chi, 2],
        gradient=True,
    )
    us = np.random.normal(scale=0.1, size=[Lx * Ly, chi, chi, 2])
    vgf_wrapper = loss_wrapper(vgf_scipy)
    optr = scipy.optimize.minimize(
        vgf_wrapper,
        K.reshape(us, [-1]),
        method="L-BFGS-B",
        jac=True,
        options={"ftol": 1e-22, "gtol": 1e-22, "maxiter": steps},
    )
    us = K.reshape(optr.x, [Lx * Ly, chi, chi, 2])
    state = get_state(us)
    state = state / K.norm(state)
    overlap0 = np.abs(K.numpy(target).reshape([1, -1]) @ u) ** 2
    overlap = np.abs(K.numpy(state).reshape([1, -1]) @ u) ** 2
    slope, intercept, r, p, slopee = scipy.stats.linregress(es, np.log(overlap))
    return (optr, slope, intercept, r, p, slopee, overlap0, overlap, vgf_wrapper.loss)

In [ ]:
get_ites(0.5, chi=32, steps=2000)

In [ ]:
get_ites_bfgs(1.5, chi=32, steps=2000)

In [ ]:
# training toward gs with history


def get_gs_training(chi=16, steps=300, which="e"):
    target = u[:, 0]
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    target = K.cast(target, tc.dtypestr)
    lst = tf.keras.optimizers.schedules.ExponentialDecay(3e-3, 1000, 0.5)
    opt = K.optimizer(tf.keras.optimizers.Adam(lst))
    us = np.random.normal(scale=0.1, size=[Lx * Ly, chi, chi, 2])
    hist = []
    ovs = []
    for i in range(steps):
        if which == "e":
            v, gs = vgf_e(us)
        elif which == "f":
            v, gs = vgf_fidelity(us, target)
        us = opt.update(gs, us)
        if i % 5 == 0:
            hist.append(v)
            state = get_state(us)
            state = state / K.norm(state)
            overlap = np.abs(K.numpy(state).reshape([1, -1]) @ u) ** 2
            ovs.append(overlap)
    return hist, ovs

In [ ]:
get_gs_training(chi=16, steps=300)

## PEPS

In [ ]:
def get_energy(us):
    state = get_state(us)
    s = state / K.norm(state)
    c = tc.Circuit(Lx * Ly, inputs=s)
    energy = tc.templates.measurements.operator_expectation(c, h)

    return K.real(energy)


def get_state(us):
    # peps Lx, Ly, chil, chir, chiu, chid, 2
    ns = {}
    for i in range(Lx):
        for j in range(Ly):
            ns[(i, j)] = tc.gates.Gate(K.copy(us[i, j]))
    for i in range(Lx):
        for j in range(Ly):
            ns[(i, j)][1] ^ ns[((i + 1) % Lx, j)][0]
            ns[(i, j)][3] ^ ns[(i, (j + 1) % Ly)][2]
        # ns[i][1]^ns[(i+1)%L][0]
    nsv = [v for k, v in ns.items()]
    m = tc.contractor(nsv, output_edge_order=[n[4] for n in nsv])
    s = K.reshape(m.tensor, [-1])
    return s


def get_fidelity(us, gs):
    state = get_state(us)
    s = state / K.norm(state)
    s = K.cast(s, tc.dtypestr)
    return -K.abs(tf.tensordot(s, gs, 1)) ** 2


vgf_fidelity = K.jit(K.value_and_grad(get_fidelity))
vgf_e = K.jit(K.value_and_grad(get_energy))

In [ ]:
# training toward gs with history


def get_gs_training(chi=4, steps=300, which="e"):
    target = u[:, 0]
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    target = K.cast(target, tc.dtypestr)
    opt = K.optimizer(tf.keras.optimizers.Adam(8e-3))
    us = np.random.normal(scale=0.1, size=[Lx, Ly, chi, chi, chi, chi, 2])
    hist = []
    ovs = []
    for i in range(steps):
        if which == "e":
            v, gs = vgf_e(us)
        elif which == "f":
            v, gs = vgf_fidelity(us, target)
        us = opt.update(gs, us)
        if i % 5 == 0:
            hist.append(v)
            state = get_state(us)
            state = state / K.norm(state)
            overlap = np.abs(K.numpy(state).reshape([1, -1]) @ u) ** 2
            ovs.append(overlap)
    return hist, ovs

In [ ]:
get_gs_training(chi=4, steps=600, which="f")

In [ ]:
def get_ites(beta, chi=4, steps=300):
    s = K.sum(
        [K.exp(-beta / 2 * es[i]) * u[:, i] for i in range(2 ** (Lx * Ly))], axis=0
    )
    target = s
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    target = K.cast(target, tc.dtypestr)
    opt = K.optimizer(tf.keras.optimizers.Adam(8e-3))  # 4e-3 8e-3 for >1
    us = np.random.normal(scale=0.1, size=[Lx, Ly, chi, chi, chi, chi, 2])
    for i in range(steps):
        v, gs = vgf_fidelity(us, target)
        us = opt.update(gs, us)
    state = get_state(us)
    state = state / K.norm(state)
    overlap0 = np.abs(K.numpy(target).reshape([1, -1]) @ u) ** 2
    overlap = np.abs(K.numpy(state).reshape([1, -1]) @ u) ** 2
    slope, intercept, r, p, slopee = scipy.stats.linregress(es, np.log(overlap))
    return (v, slope, intercept, r, p, slopee, overlap0, overlap)

In [ ]:
get_ites(1.0)

## VQE

In [ ]:
def get_energy(us):
    # mps Lx, Ly, chil, chir, chiu, chid, 2
    state = get_state(us)
    s = state / K.norm(state)
    c = tc.Circuit(Lx * Ly, inputs=s)
    energy = tc.templates.measurements.operator_expectation(c, h)

    return K.real(energy)


coord = tc.templates.graphs.Grid2DCoord(Lx, Ly)


def get_state(us):
    depth = len(us)
    c = tc.Circuit(12)
    for i in range(0, Lx * Ly - 1, 2):
        c.x(i)
        c.h(i)
        c.cx(i, i + 1)
        c.x(i + 1)
    for i in range(depth):
        c = tc.templates.blocks.Grid2D_entangling(
            c, coord, tc.gates._swap_matrix, us[i, 0]
        )
        for j in range(Lx * Ly):
            c.ry(j, theta=us[i, 1, j])
            c.rz(j, theta=us[i, 2, j])
    s = c.state()
    return s


def get_fidelity(us, gs):
    state = get_state(us)
    s = state / K.norm(state)
    return -K.abs(tf.tensordot(s, gs, 1)) ** 2


vgf_fidelity = K.jit(K.value_and_grad(get_fidelity))
vgf_e = K.jit(K.value_and_grad(get_energy))

In [ ]:
def get_ites(beta, depth=3, steps=300):
    s = K.sum(
        [K.exp(-beta / 2 * es[i]) * u[:, i] for i in range(2 ** (Lx * Ly))], axis=0
    )
    target = s
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    target = K.cast(target, tc.dtypestr)
    lst = tf.keras.optimizers.schedules.ExponentialDecay(1e-2, 2000, 0.5)
    opt = K.optimizer(tf.keras.optimizers.Adam(lst))
    us = np.random.normal(scale=0.01, size=[depth, 3, Lx * Ly * 2])
    us = K.cast(us, tc.rdtypestr)
    for i in range(steps):
        v, gs = vgf_fidelity(us, target)
        us = opt.update(gs, us)
        print(v)
    state = get_state(us)
    state = state / K.norm(state)
    overlap0 = np.abs(K.numpy(target).reshape([1, -1]) @ u) ** 2
    overlap = np.abs(K.numpy(state).reshape([1, -1]) @ u) ** 2
    slope, intercept, r, p, slopee = scipy.stats.linregress(es, np.log(overlap))
    return (v, slope, intercept, r, p, slopee, overlap0, overlap)

In [ ]:
get_ites(1.3, depth=6, steps=1000)

In [ ]:
# training toward gs with history


def get_gs_training(depth=3, steps=300, which="e"):
    target = u[:, 0]
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    target = K.cast(target, tc.dtypestr)
    lst = tf.keras.optimizers.schedules.ExponentialDecay(1e-2, 2000, 0.5)
    opt = K.optimizer(tf.keras.optimizers.Adam(lst))
    us = np.random.normal(scale=0.01, size=[depth, 3, Lx * Ly * 2])
    us = K.cast(us, tc.rdtypestr)
    hist = []
    ovs = []
    for i in range(steps):
        if which == "e":
            v, gs = vgf_e(us)
        elif which == "f":
            v, gs = vgf_fidelity(us, target)
        us = opt.update(gs, us)
        if i % 5 == 0:
            hist.append(v)
            state = get_state(us)
            state = state / K.norm(state)
            overlap = np.abs(K.numpy(state).reshape([1, -1]) @ u) ** 2
            ovs.append(overlap)
    return hist, ovs

In [ ]:
get_gs_training()

## VEC

In [ ]:
def get_energy(us):
    state = get_state(us)
    s = state / K.norm(state)
    c = tc.Circuit(Lx * Ly, inputs=s)
    energy = tc.templates.measurements.operator_expectation(c, h)
    return K.real(energy)


def get_state(us):
    us = K.cast(us, tc.dtypestr)
    s = us[: 2 ** (Lx * Ly)] + 1.0j * us[2 ** (Lx * Ly) :]
    return s


def get_fidelity(us, gs):
    state = get_state(us)
    s = state / K.norm(state)
    return -K.abs(tf.tensordot(s, gs, 1)) ** 2


vgf_fidelity = K.jit(K.value_and_grad(get_fidelity))
vgf_e = K.jit(K.value_and_grad(get_energy))

In [ ]:
def get_ites(beta, steps=300):
    s = K.sum(
        [K.exp(-beta / 2 * es[i]) * u[:, i] for i in range(2 ** (Lx * Ly))], axis=0
    )
    target = s
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    target = K.cast(target, tc.dtypestr)
    opt = K.optimizer(tf.keras.optimizers.Adam(2e-3))
    us = np.random.normal(scale=0.1, size=[2 * 2 ** (Lx * Ly)])
    us = K.cast(us, tc.rdtypestr)
    for i in range(steps):
        v, gs = vgf_fidelity(us, target)
        us = opt.update(gs, us)
    state = get_state(us)
    state = state / K.norm(state)
    overlap0 = np.abs(K.numpy(target).reshape([1, -1]) @ u) ** 2
    overlap = np.abs(K.numpy(state).reshape([1, -1]) @ u) ** 2
    slope, intercept, r, p, slopee = scipy.stats.linregress(es, np.log(overlap))
    return (v, slope, intercept, r, p, slopee, overlap0, overlap)

In [ ]:
get_ites(0.7, steps=600)

In [ ]:
# training toward gs with history


def get_gs_training(steps=300, which="e"):
    target = u[:, 0]
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    target = K.cast(target, tc.dtypestr)
    opt = K.optimizer(tf.keras.optimizers.Adam(2e-3))
    us = np.random.normal(scale=0.1, size=[2 * 2 ** (Lx * Ly)])
    us = K.cast(us, tc.rdtypestr)
    hist = []
    ovs = []
    for i in range(steps):
        if which == "e":
            v, gs = vgf_e(us)
        elif which == "f":
            v, gs = vgf_fidelity(us, target)
        us = opt.update(gs, us)
        if i % 5 == 0:
            hist.append(v)
            state = get_state(us)
            state = state / K.norm(state)
            overlap = np.abs(K.numpy(state).reshape([1, -1]) @ u) ** 2
            ovs.append(overlap)
    return hist, ovs

In [ ]:
get_gs_training(steps=200)

## VMC

In [ ]:
def resblock(m, n, x):
    y = tf.keras.layers.Dense(m, activation="relu")(x)
    y = tf.keras.layers.Dense(n, activation="relu")(y)
    return y + x


def create_complex_model(n, w=32):
    width = w * n
    inputs = tf.keras.layers.Input(shape=[n])
    lnr = resblock(width, n, inputs)
    lnr = resblock(width, n, inputs)
    lnr = resblock(width, n, inputs)
    lnr = resblock(width, n, inputs)

    phi = resblock(width, n, inputs)
    phi = resblock(width, n, inputs)
    phi = resblock(width, n, inputs)
    phi = resblock(width, n, inputs)

    lnr = 2 * tf.math.cosh(lnr)
    lnr = tf.keras.layers.Dense(1, activation=None)(lnr)

    phi = tf.keras.layers.Dense(1, activation=None)(phi)

    dtype = tf.complex128
    lnr = tf.cast(lnr, dtype=dtype)
    phi = tf.cast(phi, dtype=dtype)
    phi = 3.14159265j * phi
    outputs = lnr * tf.math.exp(phi)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


base = tf.constant(list(product(*[[0, 1] for _ in range(Lx * Ly)])))

In [ ]:
def get_ites(beta, w=32, steps=500):
    s = K.sum(
        [K.exp(-beta / 2 * es[i]) * u[:, i] for i in range(2 ** (Lx * Ly))], axis=0
    )
    target = s
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    target = K.cast(target, tc.dtypestr)

    class LRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, initial_learning_rate):
            self.initial_learning_rate = initial_learning_rate

        def __call__(self, step):
            if step < 800:
                return self.initial_learning_rate * 0.5 ** (step / 200)
            return self.initial_learning_rate * 0.06

    lst = LRSchedule(1e-3)
    opt = tf.keras.optimizers.Adam(lst)
    model = create_complex_model(Lx * Ly, w)

    @tf.function
    def value_and_grad(gs):
        with tf.GradientTape() as tape:
            state = model(base)
            state /= K.norm(state)
            state = K.reshape(state, [-1])
            fidelity = -K.abs(tf.tensordot(state, gs, 1)) ** 2
        grad = tape.gradient(fidelity, model.variables)
        return fidelity, grad

    for i in range(steps):
        v, gs = value_and_grad(target)
        opt.apply_gradients(zip(gs, model.variables))  # 1.e-5
    state = model(base)
    state = state / K.norm(state)
    overlap0 = np.abs(K.numpy(target).reshape([1, -1]) @ u) ** 2
    overlap = np.abs(K.numpy(state).reshape([1, -1]) @ u) ** 2
    slope, intercept, r, p, slopee = scipy.stats.linregress(es, np.log(overlap))
    return (v, slope, intercept, r, p, slopee, overlap0, overlap)

In [ ]:
get_ites(0.5, 32)

In [ ]:
# training toward gs with history


def get_gs_training(w=32, steps=300, which="e"):
    target = u[:, 0]
    target = K.reshape(target, [-1])
    target /= K.norm(target)
    target = K.cast(target, tc.dtypestr)

    class LRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, initial_learning_rate):
            self.initial_learning_rate = initial_learning_rate

        def __call__(self, step):
            if step < 800:
                return self.initial_learning_rate * 0.5 ** (step / 200)
            return self.initial_learning_rate * 0.06

    lst = LRSchedule(1e-3)
    opt = tf.keras.optimizers.Adam(lst)
    model = create_complex_model(Lx * Ly, w)

    if which == "f":

        @tf.function
        def value_and_grad():
            with tf.GradientTape() as tape:
                state = model(base)
                state /= K.norm(state)
                state = K.reshape(state, [-1])
                fidelity = (
                    -K.abs(tf.tensordot(state, K.cast(u[:, 0], tc.dtypestr), 1)) ** 2
                )
            grad = tape.gradient(fidelity, model.variables)
            return fidelity, grad

    elif which == "e":

        @tf.function
        def value_and_grad():
            with tf.GradientTape() as tape:
                state = model(base)
                state /= K.norm(state)
                state = K.reshape(state, [-1])
                c = tc.Circuit(Lx * Ly, inputs=state)
                energy = tc.templates.measurements.operator_expectation(c, h)
                energy = K.real(energy)
            grad = tape.gradient(energy, model.variables)
            return energy, grad

    hist = []
    ovs = []
    for i in range(steps):
        v, gs = value_and_grad()
        opt.apply_gradients(zip(gs, model.variables))  # 1.e-5
        if i % 5 == 0:
            hist.append(v)
            state = model(base)
            state = state / K.norm(state)
            overlap = np.abs(K.numpy(state).reshape([1, -1]) @ u) ** 2
            ovs.append(overlap)
    return hist, ovs

In [ ]:
get_gs_training(which="f")